In [8]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

In [3]:
load_dotenv()
KEY = os.getenv('OPENAI_API_KEY')

In [7]:
llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=.7)

C:\Users\Adeeb\AppData\Local\Temp\ipykernel_11524\2478737479.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=.7)


In [64]:
file_path = './data.txt'
TEXT = ''
with open(file_path, 'r') as file:
    TEXT = file.read()

JSON_FORMAT = {
    "1": {
        "mcq": "Ask the first question",
        "options": {
            "a": 'option 1',
            "b": 'option 2',
            "c": 'option 3',
            "d": 'option 4'
        },
        "correct_answer": "a"
    },
    "2": {
        "mcq": "Ask the second question",
        "options": {
            "a": 'option 1',
            "b": 'option 2',
            "c": 'option 3',
            "d": 'option 4'
        },
        "correct_answer": "b"
    },
    "3": {
        "mcq": "Ask the Third question",
        "options": {
            "a": 'option 1',
            "b": 'option 2',
            "c": 'option 3',
            "d": 'option 4'
        },
        "correct_answer": "d"
    }
}

NUMBER = 5
SUBJECT = 'Machine Learning'
TONE = 'Hard'
json_format_string = json.dumps(JSON_FORMAT)

quiz_generation_template = '''You are a quiz master bot. You are provided with a Input Text. 
You need to generate {number} question quiz based on the provided text for {subject} students. You need to 
set the difficulty level to {tone} level. The output should be in JSON Format as specified at the end of the prompt 

INPUT TEXT: {text}

Use the above input text to generate quiz and produce the output in JSON Format as shown below.

### OUTPUT JSON Format: 
{json_format}
'''
prompt_template = PromptTemplate(
    input_variables=['number','subject', 'tone', 'text', 'json_format'],
    template=quiz_generation_template
)

# client = OpenAI(model_name='gpt-3.5-turbo', openai_api_key=KEY)
quiz_chain = LLMChain(llm=llm, prompt=prompt_template, output_key='quiz_content', verbose=True)

# response = quiz_chain.invoke({
#     'number': NUMBER,
#     'subject': SUBJECT,
#     'tone': TONE,
#     'text': TEXT,
#     'json_format': json_format_string
# })


In [73]:
review_template = '''You are a English grammerian and writer. Read the quizzes input in {quiz_content} and analyze the complexity of the question for {subject} student.
If you think, the question doesn't meet the expectations, update the question, options and correct answer accordingly. Make sure the content is from the actual input text.
Please try to rewrite the question in a better way
### input text
{text}
#

produce the output in JSON Format as shown below.
###JSON Format: 
{json_format}
# '''

prompt_template_review = PromptTemplate(
    input_variables=['quiz_content', 'subject', 'input_text'],
    template=review_template
)

review_chain = LLMChain(llm=llm, prompt=prompt_template_review, output_key='quiz_after_review', verbose=True)
# response = quiz_chain.invoke({
#     'number': NUMBER,
#     'subject': SUBJECT,
#     'tone': TONE,
#     'text': TEXT,
#     'json_format': json_format_string
# })

sequential_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=['number','subject', 'tone', 'text', 'json_format'],
                                    output_variables=['quiz_content','quiz_after_review'])

with get_openai_callback() as callback:
    response = sequential_chain.invoke({
        'number': NUMBER,
        'subject': SUBJECT,
        'tone': TONE,
        'text': TEXT,
        'json_format': json_format_string
    })
print(callback)
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are a quiz master bot. You are provided with a Input Text. 
You need to generate 5 question quiz based on the provided text for Machine Learning students. You need to 
set the difficulty level to Hard level. The output should be in JSON Format as specified at the end of the prompt 

INPUT TEXT: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structu

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:
You are a English grammerian and writer. Read the quizzes input in {
    "1": {
        "mcq": "Who coined the term 'machine learning' in 1959?",
        "options": {
            "a": "Donald Hebb",
            "b": "Raytheon Company",
            "c": "Tom M. Mitchell",
            "d": "Arthur Samuel"
        },
        "correct_answer": "d"
    },
    "2": {
        "mcq": "Which book introduced a theoretical neural structure formed by certain interactions among nerve cells?",
        "options": {
            "a": "The Organization of Behavior",
            "b": "Learning Machines",
            "c": "Computing Machinery and Intelligence",
            "d": "Research into Machine Learning"
        },
        "correct_answer": "a"
    },
    "3": {
        "mcq": "In which year was a report given on using teaching strategies for recognizing characters from a computer terminal?",
        "options": {
            "a": "1949",
            "b": "

{'number': 5,
 'subject': 'Machine Learning',
 'tone': 'Hard',
 'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.

In [75]:
print(f'Prompt Tokens:{callback.prompt_tokens}')
print(f'Completion Tokens:{callback.completion_tokens}')
print(f'Total Cost:{callback.total_cost}')


mcq_json = json.loads(response['quiz_after_review'])
mcq_json

Prompt Tokens:2254
Completion Tokens:861
Total Cost:0.005103


{'1': {'mcq': "Who is credited with coining the term 'machine learning' in 1959?",
  'options': {'a': 'Donald Hebb',
   'b': 'Raytheon Company',
   'c': 'Tom M. Mitchell',
   'd': 'Arthur Samuel'},
  'correct_answer': 'd'},
 '2': {'mcq': 'Which book introduced a theoretical neural structure based on interactions among nerve cells?',
  'options': {'a': 'The Organization of Behavior',
   'b': 'Learning Machines',
   'c': 'Computing Machinery and Intelligence',
   'd': 'Research into Machine Learning'},
  'correct_answer': 'a'},
 '3': {'mcq': 'When was a report presented on using teaching strategies for recognizing characters from a computer terminal?',
  'options': {'a': '1949', 'b': '1960', 'c': '1973', 'd': '1981'},
  'correct_answer': 'd'},
 '4': {'mcq': 'How does Tom M. Mitchell define machine learning algorithms?',
  'options': {'a': 'Learning from experience with respect to tasks and performance',
   'b': 'Recognizing patterns using reinforcement learning',
   'c': 'Classifying dat

In [76]:
questions = []
for index, item in mcq_json.items():
    mcq = item['mcq']
    
    options = ' | '.join(
        [f"{option}: {option_value}" for option, option_value in item['options'].items()]
    )
    correct_answer = item['correct_answer']
    questions.append({
        'mcq': mcq,
        'options': options,
        'answer': correct_answer
    })
questions


[{'mcq': "Who is credited with coining the term 'machine learning' in 1959?",
  'options': 'a: Donald Hebb | b: Raytheon Company | c: Tom M. Mitchell | d: Arthur Samuel',
  'answer': 'd'},
 {'mcq': 'Which book introduced a theoretical neural structure based on interactions among nerve cells?',
  'options': 'a: The Organization of Behavior | b: Learning Machines | c: Computing Machinery and Intelligence | d: Research into Machine Learning',
  'answer': 'a'},
 {'mcq': 'When was a report presented on using teaching strategies for recognizing characters from a computer terminal?',
  'options': 'a: 1949 | b: 1960 | c: 1973 | d: 1981',
  'answer': 'd'},
 {'mcq': 'How does Tom M. Mitchell define machine learning algorithms?',
  'options': 'a: Learning from experience with respect to tasks and performance | b: Recognizing patterns using reinforcement learning | c: Classifying data based on models | d: Making predictions for future outcomes',
  'answer': 'a'},
 {'mcq': 'What are the primary obj

In [70]:
questions_df = pd.DataFrame(questions)
questions_df.head()


,mcq,options,answer
0,Who first introduced the term machine learning...,a: Donald Hebb | b: Raytheon Company | c: Tom ...,d
1,What was the name of the experimental 'learnin...,a: Cybertron | b: Nilsson's book | c: Duda and...,a
2,Who provided a formal definition of the algori...,a: Alan Turing | b: Warren McCulloch | c: Rayt...,d
3,What is one of the main objectives of modern-d...,a: To invent new algorithms | b: To classify d...,b
4,What did Alan Turing propose in his paper 'Com...,a: Can machines think? | b: Can machines do wh...,b


In [77]:
questions_df.to_csv('./quiz.csv')